In [ ]:
!pip install neuron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 51.0 MB/s eta 0:00:00


In [ ]:
!pip install pymc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from neuron import h
h.load_file('stdrun.hoc')

# Define the experimental data (observations)
# This can be obtained from experimental recordings
obs_time = np.linspace(0, 50, 1000)
obs_membrane = np.sin(obs_time) # replace with actual experimental data

# Define the summary statistics
# We will use the mean and standard deviation of the simulated membrane potential


class HodgkinHuxleyModel:
    def __init__(self):
        # Create the soma section
        self.soma = h.Section(name='soma')
        self.soma.insert('hh')
        
        # Define the soma geometry and biophysics
        self.soma.L = 30 # um
        self.soma.diam = 30 # um
        self.soma.Ra = 100 # ohm-cm
        self.soma.cm = 1 # uF/cm^2
        self.soma.insert('pas')
        self.soma.g_pas = 1.0 / 15000.0
        self.soma.e_pas = -65
        
    def set_conductances(self, gnabar, gkbar):
        self.soma.gnabar_hh = gnabar
        self.soma.gkbar_hh = gkbar
        
    def simulate(self, time):
        # Set up the simulation
        stim = h.IClamp(self.soma(0.5))
        stim.delay = 10 # ms
        stim.dur = 1 # ms
        stim.amp = 0.1 # nA
        
        rec = h.Vector()
        rec.record(self.soma(0.5)._ref_v)
        
        # Run the simulation
        h.finitialize(-65)
        h.continuerun(time)
        
        # Convert the voltage recording to a numpy array
        membrane_potential = np.array(rec)



        
        return membrane_potential
def compute_summary_stats(sim_membrane):
    return np.mean(sim_membrane), np.std(sim_membrane)

# Define the distance function
# We will use the Euclidean distance between the summary statistics of the observed and simulated data
def compute_distance(obs_stats, sim_stats):
    return np.sqrt(np.sum((obs_stats - sim_stats)**2))

# Define the ABC algorithm
def abc_algorithm(model, obs_membrane, obs_time, n_particles, n_generations, epsilon):
    # Initialize the particle parameters randomly
    particles = []
    for i in range(n_particles):
        particles.append({'gnabar': random.uniform(0, 1), 'gkbar': random.uniform(0, 1)})
    
    # Run the generations
    for generation in range(n_generations):
        # Simulate the particles
        for particle in particles:
            # Set the parameters
            model.set_conductances(particle['gnabar'], particle['gkbar'])
            
            # Run the simulation and plot the membrane potential
            sim_membrane = model.simulate(obs_time)
            plt.plot(obs_time, sim_membrane)
            
            # Compute the summary statistics
            sim_stats = compute_summary_stats(sim_membrane)
            
            # Compute the distance
            dist = compute_distance(compute_summary_stats(obs_membrane), sim_stats)
            
            # If the distance is less than epsilon, we have found a good particle
            if dist < epsilon:
                return particle
            
        # Select the best particles to generate the next generation
        particles = select_particles(particles)
        
    # If we have not found a good particle after n_generations, return None
    return None


def select_particles(particles):
    # Sort the particles by increasing distance
    particles = sorted(particles, key=lambda p: p['dist'])
    
    # Keep the best particles
    n_particles = len(particles)
    n_keep = int(n_particles / 2)
    particles = particles[:n_keep]
    
    # Create new particles by perturbing the best particles
    new_particles = []
    for particle in particles:
        new_particle = {}
        for key, value in particle.items():
            new_particle[key] = value + random.gauss(0, 0.1)
        new_particles.append(new_particle)
        
    # Combine the best particles with the new particles
    particles = particles + new_particles
    
    return particles


# Define the simulation function
def simulate_membrane(model, time):
    # Run the simulation
    h.tstop = time[-1]
    h.run()
    membrane_potential = np.array(model.soma(0.5)._ref_v)
    
    # Plot the membrane potential over time
    plt.plot(np.array(time), np.array(membrane_potential))
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Potential (mV)')
    plt.show()
    
    # Return the simulated membrane potential
    return membrane_potential

